In [2]:
import numpy as np
import cv2, time, os
from matplotlib import pyplot as plt
import pandas as pd


In [3]:
im_source = cv2.imread('C:\\Users\\user\\friend_photos\\main_photo.jpg',1)


detector  = cv2.KAZE_create()
matcher = cv2.BFMatcher(cv2.NORM_L1)

kp_src, des_src = detector.detectAndCompute(im_source, None)

# if len(kp_sch) < 2 or len(kp_src) < 2:
#     raise _robot_error("Not enough feature points in input images !")
    


In [5]:
def find_metrics(path_im_search,detector,matcher,kp_src, des_src):
#     path_im_search - str

    im_search  = cv2.imread(path_im_search,1)
    kp_sch, des_sch = detector.detectAndCompute(im_search, None)
    start_time = time.time()
    matches = matcher.knnMatch(des_sch, des_src, k=2)
    all_matches = len(matches) * 2
    good = []

    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    # Set the source image can not have duplicate points        
    good_diff, diff_good_point = [], [[]]

    for m in good:
        diff_point = [int(kp_src[m.trainIdx].pt[0]), int(kp_src[m.trainIdx].pt[1])]
        if diff_point not in diff_good_point:
            good_diff.append(m)
            diff_good_point.append(diff_point)

    good = good_diff
    good_matches = len(good)

    end_time = time.time()
    result_time = round(end_time - start_time, 4)

    error_all_matches_list = []
    for m in np.array(matches).ravel().tolist():
        error_all_matches_list.append(m.distance)
    error_all_matches = round(np.array(error_all_matches_list).mean(), 4)

    error_good = []
    for m in good:
        error_good.append(m.distance)
    error_good = round(np.array(error_good).mean(), 4)

    size = im_search.shape

    return all_matches, good_matches, error_all_matches, error_good, size, result_time



In [6]:
%%time
all_metrics = []
i = 1
for filename in os.listdir('C:\\Users\\user\\friend_photos\\Library'):
    i += 1
    temp_list = list(find_metrics('C:\\Users\\user\\friend_photos\\Library\\' + filename,detector,matcher,kp_src, des_src))
    temp_list.insert(0, filename)
    all_metrics.append(temp_list)

df = pd.DataFrame(all_metrics, columns=['name', 'all_matches', 'true_matches', 
                                        'error_all_matches', 'error_true_matches', 'size', 'time'])


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice.
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: Mean of empty slice.


Wall time: 5min 31s


In [7]:
df.to_csv( 'C:\\Users\\user\\friend_results\\metrics.csv') 

In [8]:
df.head(10)

,name,all_matches,true_matches,error_all_matches,error_true_matches,size,time
0,1.jpg,1618,95,2.2630,1.5432,"(1276, 1276, 3)",0.0309
1,2.jpg,2110,43,2.2860,1.6708,"(1276, 1276, 3)",0.0419
2,3.jpg,1012,2,2.8706,2.2580,"(1276, 956, 3)",0.0179
3,4.jpg,1812,184,2.2742,1.4354,"(1276, 1276, 3)",0.0349
4,5.jpg,74,1,2.9402,1.7035,"(1276, 1276, 3)",0.0020
5,IMG_6806.JPG,4098,120,2.3160,1.7674,"(4032, 3024, 3)",0.0748
6,IMG_6807.JPG,2472,185,2.0223,1.4599,"(4032, 3024, 3)",0.0479
7,IMG_6808.JPG,4094,134,2.2280,1.5366,"(4032, 3024, 3)",0.0808
8,IMG_6809.JPG,5488,56,2.2016,1.5067,"(4032, 3024, 3)",0.0997
9,IMG_6810.JPG,4384,81,2.2882,1.5619,"(4032, 3024, 3)",0.0788


In [9]:
source_keys = cv2.drawKeypoints(im_source,kp_src , None)
cv2.namedWindow("Benchmark", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Benchmark", 600, 600)
cv2.imshow("Benchmark", source_keys)

cv2.imwrite('C:\\Users\\user\\friend_results\\source_keys.jpg', source_keys)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
#all matches for source and 4 example photos
i=0

for filename in os.listdir('C:\\Users\\user\\friend_photos\\photos_for_example'):
    
    i+=1
    
    im_search = cv2.imread('C:\\Users\\user\\friend_photos\\photos_for_example\\'+filename, 1)
    im_source = cv2.imread('C:\\Users\\user\\friend_photos\\main_photo.jpg',1)

    kp_src, des_src = detector.detectAndCompute(im_source, None)
    kp_sch, des_sch = detector.detectAndCompute(im_search, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_L2)

    matches = matcher.knnMatch(des_src, des_sch, k=2)
    
    matches = np.array(matches).ravel().tolist()
    matching_result = cv2.drawMatches(im_source, kp_src, im_search, kp_sch, matches, None)

    cv2.namedWindow("Matches", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Matches", 1200, 600)
    cv2.imshow("Matches", matching_result)

    cv2.imwrite('C:\\Users\\user\\friend_results\\all_matches\\match' + str(i) + '.jpg', matching_result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [19]:
#true matches for source and 4 example photos
i=0

for filename in os.listdir('C:\\Users\\user\\friend_photos\\photos_for_example'):
    
    i+=1
    im_search = cv2.imread('C:\\Users\\user\\friend_photos\\photos_for_example\\'+filename, 1)
    im_source = cv2.imread('C:\\Users\\user\\friend_photos\\main_photo.jpg',1)

    kp_src, des_src = detector.detectAndCompute(im_source, None)
    kp_sch, des_sch = detector.detectAndCompute(im_search, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_L2)

    matches = matcher.knnMatch(des_src, des_sch, k=2)
    
    good = []

    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    # Set the source image can not have duplicate points        
    good_diff, diff_good_point = [], [[]]
    for m in good:
        diff_point = [int(kp_src[m.trainIdx].pt[0]), int(kp_src[m.trainIdx].pt[1])]
        if diff_point not in diff_good_point:
            good_diff.append(m)
            diff_good_point.append(diff_point)

    good = good_diff

    matching_result = cv2.drawMatches(im_source, kp_src, im_search, kp_sch, good, None)

    cv2.namedWindow("Matches", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Matches", 1200, 600)
    cv2.imshow("Matches", matching_result)

    cv2.imwrite('C:\\Users\\user\\friend_results\\true_matches\\match' + str(i) + '.jpg', matching_result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()